In [ ]:
import pandas as pd
import spacy
import os
import networkx as nx
from collections import defaultdict
import csv
from tqdm import tqdm 
import re
import matplotlib.pyplot as plt
from spacy.pipeline import EntityRuler


In [ ]:
nlp = spacy.load("en_core_web_trf") 

In [ ]:
docs = []

directory= 'data4/clean/'

whole = ""

for filename in os.listdir(directory):
    if filename.endswith('.txt'):  
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            whole += content + "\n\n"  


print(whole)  
nlp.max_length = len(whole) + 100 

In [ ]:
file_path = './manchester_data.csv'
df = pd.read_csv(file_path)


street_names = df['final_text'].dropna().unique()


whole_text = whole


def is_valid_street_name(name):
    if len(name) > 5 and len(name) < 50:
        if re.match("^[A-Za-z0-9 .,'&-]+$", name):
            return True
    return False

valid_street_names = [name for name in street_names if is_valid_street_name(name)]